In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen_train = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, 
                               brightness_range = (0.1, 0.5), horizontal_flip=True)

train_data = gen_train.flow_from_directory("/content/drive/MyDrive/Dataset",
                                           target_size = (224, 224), batch_size = 32, class_mode="categorical")

Found 3350 images belonging to 8 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

# here i'm going to take input shape, weights and bias from imagenet and include top False means
# i want to add input, flatten and output layer by my self

vgg16 = VGG16(input_shape = (224, 224, 3), weights = "imagenet", include_top = False)

In [ ]:
for layer in vgg16.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras import layers

x = layers.Flatten()(vgg16.output)

In [ ]:
# now let's add output layers or prediction layer

prediction = layers.Dense(units = 8, activation="softmax")(x)

# creating a model object

model = tf.keras.models.Model(inputs = vgg16.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics =["accuracy"])

result = model.fit(train_data, epochs = 28, steps_per_epoch=len(train_data))

Epoch 1/28
105/105 [==============================] - 923s 9s/step - loss: 1.2968 - accuracy: 0.5340
Epoch 2/28
105/105 [==============================] - 88s 833ms/step - loss: 0.8788 - accuracy: 0.6797
Epoch 3/28
105/105 [==============================] - 88s 833ms/step - loss: 0.7818 - accuracy: 0.7125
Epoch 4/28
105/105 [==============================] - 88s 841ms/step - loss: 0.7667 - accuracy: 0.7149
Epoch 5/28
105/105 [==============================] - 89s 842ms/step - loss: 0.6693 - accuracy: 0.7606
Epoch 6/28
105/105 [==============================] - 86s 821ms/step - loss: 0.6644 - accuracy: 0.7573
Epoch 7/28
105/105 [==============================] - 86s 813ms/step - loss: 0.5863 - accuracy: 0.7904
Epoch 8/28
105/105 [==============================] - 87s 827ms/step - loss: 0.6200 - accuracy: 0.7800
Epoch 9/28
105/105 [==============================] - 88s 839ms/step - loss: 0.5364 - accuracy: 0.8119
Epoch 10/28
105/105 [==============================] - 90s 860ms/step - los

In [ ]:
from tensorflow.keras.preprocessing import image
output_class = ["batteries", "carrot","clothes", "e-waste", "light blubs","potato","rose","tomato"]
def waste_prediction(new_image):
  test_image = image.load_img(new_image, target_size = (224,224))
  plt.axis("off")
  plt.imshow(test_image)
  plt.show()
 
  test_image = image.img_to_array(test_image) / 255
  test_image = np.expand_dims(test_image, axis=0)

  predicted_array = model.predict(test_image)
  predicted_value = output_class[np.argmax(predicted_array)]
  predicted_accuracy = round(np.max(predicted_array) * 100, 2)

  print("Your waste material is ", predicted_value)

In [ ]:
model.save("newmodel.h5")

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
!pwd
!ls

2.7.0
/content
9v-battery-isolated-white-sign-energy-24846904.jpg  newmodel.h5   sample_data
burnt-light-bulb-12925025.jpg			    potato2.jpeg  tomato2.jpeg
drive						    rose0.jpeg
My_TFlite.tflite				    rose9.jpeg


In [ ]:
TF_LITE_MODEL_FILE_NAME = "tf_lite.tflite"

In [ ]:
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp_z69tzjp/assets


INFO:tensorflow:Assets written to: /tmp/tmp_z69tzjp/assets
